<a href="https://colab.research.google.com/github/anthonymelson/portfolio/blob/master/FrozenLakeFinal4x4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement Learning in Python: Solving Frozen Lake 4x4 w Q-Learning

Frozen Lake is a classic problem in reinforcement learning.  It is considered solved if the agent is able to win 78% of the time or more in 100+ trials.  Traditionally, this problem is solved with value iteration.  Lets see how Q-learning fairs!

### Import Packages

In [ ]:
!pip install gym[all]
!pip install box2d box2d-kengz

In [14]:
import gym
import matplotlib.pyplot as plt
import numpy as np

### FrozenLake-v0 (4x4)

FrozenLake-v0 is a simulated environment with 16 states.  The objective is to go from a set starting point, to a finishing point (at the top left and bottom right of a square grid) without falling into "holes" (of which there are 4).

Though there is an obvious solution to the deterministic version where you always go where you want, the stochastic version where you only have a 1/3 chance of going to the cell you select (the rest of which being distributed among the other 3 directions and wieghted to the two closest) is much more difficult.

Below is a map of the environment as it is designed in OpenAI Gym.

![map](https://twice22.github.io/images/rl_series/frozenlake.png)

### Create the Environment

Gym offers many standardized environments in many different categories.  This example is from the 'toy-text' class.  It is a classical problem and is perfect to get started with.

In [15]:
env = gym.make('FrozenLake-v0')
env.reset()

0

### Examine the Environment

Gym's environment class has a small number of methods.  They are:

**step()** : which takes an action as an input, and returns new state, reward, done (telling if the game is over), and info (a dict for debugging)

**observation_space()** : which returns the shape and type (continuous, discrete, Box) of the observation space (states) and .n which gives the number of states

**action_space()** : which returns the shape of the action space (choices) and .n which gives the number of actions

**render()** : which takes 'human', 'rgb', and 'ascii' as inputs, and returns either a video, text, or dict as output that shows how the game-world is functioning

In [16]:
print(f"Observation Space: {env.observation_space}")
print(f"Number of States: {env.observation_space.n}")
print(f"Action Space: {env.action_space}")
print(f"Number of Actions: {env.action_space.n}")
print()
print("View of Text-Map:")
env.render()

Observation Space: Discrete(16)
Number of States: 16
Action Space: Discrete(4)
Number of Actions: 4

View of Text-Map:

SFFF
FHFH
FFFH
HFFG


### Define Necessary Function
We will create two functions:

**act**: This will select actions from the action space.  It will mix its selections between random actions and actions based on the best policy known at the time of the decision.  As the agent plays through more episodes, the probability of random action will decrease (an exploration strategy called "epsilon decay").

**learn**: This will apply Q-learning.  Meaning, it will update a Q-table that maps every possible state/action pair to a reward.  By updating it based on rewards from previous actions, the table will become ever better at approximating the actual value of specific actions.  In doing so, the table will converge on the "optimal policy".

In [17]:
def act(state):
  if np.random.uniform(0,1) < epsilon:
    action = env.action_space.sample()  
  else:
    action = np.argmax(q_table[state,:]) 
  return action

In [18]:
def learn(state, new_state, reward, action):

  q_table[state, action] = q_table[state, action] + learning_rate * (reward + gamma * np.max(q_table[new_state,:]) - q_table[state, action])

### Initialize Global Values

**gamma** = discount rate (how much less weight to give to future rewards)

**learning_rate** = amount of affect new rewards have on old mappings during each update round

**episode** = the number of times the game will be started over again during agent training

**epsilon** = the probability that a random action will be taken in each step

**max_steps** = number of steps the agent takes in each episode



In [19]:
gamma = 0.95
learning_rate = 0.1
episodes = 15000
epsilon = 1.0
max_steps = 99

### Manage Epsilon Decay

The decay rate is proportional to the number of episodes so it will reach is minimum (0.1) towards the end of training.  It gets slightly smaller in each round and thus allows for less random actions to be taken.

The purpose is to manage the explore/exploit trade-off, which is essential in RL.  The randomness in the begining allows the agent to explore and learn about its environment so it can evaluate its many options before it gets set on one.  

As it decreases, the agent can begin doing what it has a pretty good idea works, and focus its exploration on refining those core strategies.

Finally, when it runs out, it can exploit what it knows to maximize its gains.

In [20]:
start_epsilon_decaying = 1
end_epsilon_decaying = episodes // 2
epsilon_decay_value = epsilon/(end_epsilon_decaying - start_epsilon_decaying)
epsilon_min = 0.01

### Create Q-Table

The Q-table maps each state to it possible actions and learns the optimal mappings if updated properly.  In the case of FrozenLake-v0, there are 4 discrete options (up, down, right, left) and 16 states (4x4 square), thus the Q-table will be 4x16 or 64 spaces.

This can be created by getting the number of obs and actions, and turning them into an n x m vector.

In [21]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

### Run the Q-Learning Algorithm

This is where the Q-table is optimized by the rewards gained from the agents exploration/exploitation of the environment.  The agent plays the game, and the table is updated based on the rewards that result from specific actions in specific states.

In [22]:
epsilon = 0.99
e = 0
terminal_state = []
for i in range(episodes):
  state = env.reset()
  t = 0

  while t < max_steps:

      action = act(state) # Select Action
      new_state, reward, done, info = env.step(action) # Take Action
      reward = reward
      learn(state, new_state, reward, action) # Get New_State and Reward

      state = new_state # Update State
      t = t + 1

      if done == True:
        break
  e = e + 1
  terminal_state.append(state)
 
  if end_epsilon_decaying >= e >= start_epsilon_decaying:
    if epsilon > epsilon_min:
      epsilon -= epsilon_decay_value
env.close()

### View Q-Table

In [23]:
print(q_table)

[[0.17596152 0.15464652 0.15368711 0.15297813]
 [0.07931432 0.09119606 0.09582933 0.14541445]
 [0.13098817 0.09723732 0.08964217 0.0898448 ]
 [0.06879116 0.06872574 0.06424997 0.09633005]
 [0.20624316 0.13098031 0.12443471 0.12513763]
 [0.         0.         0.         0.        ]
 [0.10841471 0.0437884  0.09200703 0.0458038 ]
 [0.         0.         0.         0.        ]
 [0.19012222 0.1815462  0.20665824 0.27173   ]
 [0.15328256 0.35346571 0.28020713 0.26414851]
 [0.34376092 0.23803768 0.22547474 0.16984093]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.21585321 0.28962942 0.49030649 0.27073966]
 [0.53705548 0.84664011 0.56509102 0.45630285]
 [0.         0.         0.         0.        ]]


### Test Learner

Runs for 100 episodes with the trained agent and counts how many times the it wins.  It wins 82 times in this round of 100, which is enough to "pass" (according to the original problem definition for Frozen Lake 4x4 which is 78% over 100 trials).

It prints the terminal frame of each game (18 ending in holes, and 82 ending at the goal)

In [45]:
win = []
for i in range(100):
  state = env.reset()
  moves = 0

  for _ in range(300):
    action = np.argmax(q_table[state])
    state_new, reward, done, _ = env.step(action)

    #env.render()
    state = state_new
    moves = moves + 1

    if done:
      break
  win.append(reward)
  
  # Render last 5 games
  if i >= 95:
    env.render()
  #print(moves)
print(sum(win))

  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
82.0


## Conclusion

With Q-learning we were able to solve the stochastic frozen lake 4x4 problem.  This indicates that our agent was not only able to account for the small number of paths, but also for the randomness built into the game.  This level of strategy is what AI (specifically RL) is all about.